In [1]:
from repo import cloner,filter,chunker,embedding

git_clone_dir = '/Users/harshpatel/Desktop/CodeBase Chat ( RAG + LLM )/git_clone'

GIT_REPO_PATH = "https://github.com/langchain-ai/langchain"


/opt/anaconda3/envs/rag-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cloner.clone_git_repo(GIT_REPO_PATH,git_clone_dir)

Copying repositery and storing it into git_clone folder....
Git Repositery Suceessfully Cloned and Saved in git_clone folder


In [3]:
filter.filter_files(git_clone_dir)

--------------------------------------------------------------------------------------------------------------------
Total files :: 2953 
Time took to filtering -----   0.04058408737182617 Seconds
Total got 2581 files after filtering
Remaining files : ['README.md', 'AGENTS.md', 'CLAUDE.md', 'SECURITY.md', 'README.md', 'README.md', 'extended_testing_deps.txt', 'README.md', '__init__.py', 'test_html_security.py', 'conftest.py', '__init__.py', 'test_text_splitters.py', 'test_nlp_text_splitters.py', '__init__.py', 'test_text_splitter.py', 'test_compile.py', 'nltk.py', 'jsx.py', 'html.py', '__init__.py', 'konlpy.py', 'markdown.py', 'python.py', 'character.py', 'sentence_transformers.py', 'spacy.py', 'latex.py', 'json.py', 'base.py', 'check_imports.py', 'extended_testing_deps.txt', 'README.md', '__init__.py', 'conftest.py', 'test_setup.py', 'prompt_file.txt', 'test_pydantic_serde.py', 'test_sys_info.py', 'test_prompt_values.py', 'pydantic_utils.py', 'test_imports.py', '__init__.py', 'test_gl

In [4]:
chunks = chunker.chunker(git_clone_dir)

📄 Loaded 4 text/code files successfully
📄 Loaded 5 text/code files successfully
📄 Loaded 6 text/code files successfully
📄 Loaded 8 text/code files successfully
📄 Loaded 9 text/code files successfully
📄 Loaded 13 text/code files successfully
📄 Loaded 17 text/code files successfully
📄 Loaded 17 text/code files successfully
📄 Loaded 30 text/code files successfully
📄 Loaded 30 text/code files successfully
📄 Loaded 31 text/code files successfully
📄 Loaded 33 text/code files successfully
📄 Loaded 34 text/code files successfully
📄 Loaded 50 text/code files successfully
📄 Loaded 52 text/code files successfully
📄 Loaded 57 text/code files successfully
📄 Loaded 60 text/code files successfully
📄 Loaded 64 text/code files successfully
📄 Loaded 72 text/code files successfully
📄 Loaded 74 text/code files successfully
📄 Loaded 79 text/code files successfully
📄 Loaded 85 text/code files successfully
📄 Loaded 87 text/code files successfully
📄 Loaded 92 text/code files successfully
📄 Loaded 94 text/code

In [5]:
texts = [chunk.page_content for chunk in chunks]
embeddings = embedding.convert_to_embeddings(texts)

Embedding dimention is 384
Model is converting text into embeddings....
4962 Embeddings generated.


In [6]:
print(f"Length of chunks      : {len(chunks)}")
print(f"Length of embeddings  : {len(embeddings)}")

Length of chunks      : 4962
Length of embeddings  : 4962


In [9]:
from repo import vector_db

vector_store = vector_db.VectorStore()
vector_store.add_documents(chunks,embeddings)

VectorStore Initialised. Collection : coding_files
Documents added successfully to vector store


In [52]:
from typing import List, Dict, Any

class RAGRetriever():
    def __init__(self, vector_db, embedder, top_k: int = 8):
        """
        vector_db : Chroma client instance
        embedder  : Embedding model with .embed_query(text)
        top_k     : Number of chunks to return
        """
        self.vector_db = vector_db
        self.embedder = embedder
        self.top_k = top_k

    def _filter_documents(self, docs: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Re-rank docs by applying heuristics:
        - Boost MD files (.md)
        - Remove internal / noisy code like version/constants
        """

        refined = []
        for d in docs:
            source = (d["metadata"].get("source", "")).lower()

            # Base score from vector DB
            score = d["similarity_score"]

            # 🚀 Boost markdown documentation relevance
            if source.endswith(".md"):
                score -= 0.20  # improvement boost

            # ❌ Remove noisy or low information files
            if (
                "version.py" in source
                or "constants" in source
                or source.endswith("__init__.py")
                or "tests" in source
            ):
                continue  # skip this doc completely

            # Attach new score back
            d["similarity_score"] = score
            refined.append(d)

        # Sort by refined score (lower = better)
        refined.sort(key=lambda x: x["similarity_score"])

        return refined

    def retrieve(self, query: str) -> List[Dict[str, Any]]:
        """Search vector DB and return ranked code/doc chunks with metadata"""
        print(f"🔍 Retrieving top {self.top_k} chunks for query: {query}")

        query_embedding = self.embedder.embed_query(query)

        results = self.vector_db.similarity_search_with_score(
            query_embedding, k=self.top_k + 5  # extra docs for filtering
        )

        # results → List[(Document, score)]
        formatted_results = []
        for doc, score in results:
            metadata = doc.metadata or {}
            formatted_results.append({
                "id": metadata.get("id", None),
                "content": doc.page_content,
                "metadata": metadata,
                "similarity_score": float(score),
            })

        # 3️⃣ Filter weak/noisy chunks
        final_docs = self._filter_documents(formatted_results)[:self.top_k]

        print(f"📌 Retrieved {len(final_docs)} usable chunks")

        return final_docs


In [53]:
retriever = RagRetriever(vector_store,embedding)

In [54]:
retriever.retriever('Why should someone build on LangChain?')

Embedding dimention is 384
Model is converting text into embeddings....
1 Embeddings generated.
Successfully retrieved similar content from documents.


[{'id': '_5bac955b_102',
  'content': "# 🦜🍎️ LangChain Core\n\n[![PyPI - Version](https://img.shields.io/pypi/v/langchain-core?label=%20)](https://pypi.org/project/langchain-core/#history)\n[![PyPI - License](https://img.shields.io/pypi/l/langchain-core)](https://opensource.org/licenses/MIT)\n[![PyPI - Downloads](https://img.shields.io/pepy/dt/langchain-core)](https://pypistats.org/packages/langchain-core)\n[![Twitter](https://img.shields.io/twitter/url/https/twitter.com/langchainai.svg?style=social&label=Follow%20%40LangChainAI)](https://twitter.com/langchainai)\n\nLooking for the JS/TS version? Check out [LangChain.js](https://github.com/langchain-ai/langchainjs).\n\nTo help you ship LangChain apps to production faster, check out [LangSmith](https://smith.langchain.com).\n[LangSmith](https://smith.langchain.com) is a unified developer platform for building, testing, and monitoring LLM applications.\n\n## Quick Install\n\n```bash\npip install langchain-core\n```\n\n## 🤔 What is this?\

In [ ]:

# Groq_key = <GROQ_API>
dfd

In [56]:

def rag_simple(retriever,query,llm):
    result = retriever.retriever(query,5)

    context = "\n\n".join([doc['content'] for doc in result]) if result else ""
    prompt = """Using below given context give me answer based on that
        context : {context}
        query : {query}
        answer : ..."""
    
    answer = llm.invoke([prompt.format(context = context,query = query)])
    return answer.content

In [57]:
# def rag_advanced(retriever,query,llm):
def format_context(results):
    context_blocks = []

    for item in results:  # results: [(doc, score)] or [doc]
        if isinstance(item, tuple):
            doc, score = item
        else:
            doc = item

        meta = doc['metadata']

        block = (
            f"[file_path: {meta.get('source')} | "
            f"lines {meta.get('line_start')}–{meta.get('finish_line')}]"
            "\n"
            f"{doc['content'].strip()}"
        )
        context_blocks.append(block)

    return "\n\n---\n\n".join(context_blocks)


def build_prompt(user_query, context):
    return f"""
        You are CodeBase Chat — an AI expert on this repository.
        Use ONLY the context below to answer.

        CITE the file paths and line numbers for every key detail.
        If answer not found → say “Not enough info in repo.”

        Question:
        {user_query}

        Context:
        {context}

        Answer:
        """



In [62]:
user_query = "Explain versioning in LangChain and why it is important."
retrieved_docs = retriever.retriever(user_query)  # Your retriever call

context = format_context(retrieved_docs[:4])  # limit top 4
prompt = build_prompt(user_query, context)

llm = ChatGroq(api_key=Groq_key,model="llama-3.3-70b-versatile",temperature=0.1,max_tokens=1024)

# Send to LLM (pseudo-code depending on provider)
response = llm.invoke(prompt)
print(response)

Embedding dimention is 384
Model is converting text into embeddings....
1 Embeddings generated.
Successfully retrieved similar content from documents.
content='Versioning in LangChain is explained in the context as a way to manage different versions of the LangChain ecosystem. \n\nThe importance of versioning is highlighted in [file_path: /Users/harshpatel/Desktop/CodeBase Chat ( RAG + LLM )/git_clone/libs/core/README.md | lines 34-36] where it states that the LangChain ecosystem is committed to a stable versioning scheme, and will communicate any breaking changes with advance notice and version bumps.\n\nThe versioning policy can be found in [file_path: /Users/harshpatel/Desktop/CodeBase Chat ( RAG + LLM )/git_clone/libs/core/README.md | line 41] which points to the [Versioning](https://docs.langchain.com/oss/python/versioning) policy.\n\nSpecific versions are mentioned in the following files:\n- [file_path: /Users/harshpatel/Desktop/CodeBase Chat ( RAG + LLM )/git_clone/libs/core/lan